In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




In [ ]:
#Creating the training data 
import zipfile
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 

data = pd.read_csv('/kaggle/input/painter-by-numbers/train_info.csv')
relevant_style = 'Impressionism'
relevant_data = data[data['style'] == relevant_style]

Dataset = 'painter-by-numbers/train'

zip_path = '../input/painter-by-numbers/train.zip'
relevant_files = [str("train/" + i) for i in relevant_data['filename']]





In [ ]:
Image.MAX_IMAGE_PIXELS = None
training_data = None
count = 0
with zipfile.ZipFile(zip_path) as myzip:
    fileInitial = myzip.open(relevant_files[0])
    img = Image.open(fileInitial)
    img = img.resize((200, 200))
    training_data = np.array([np.asarray(img)])
    for file in relevant_files[1:]:
        
        with myzip.open(file) as myfile:
            print(count)
            count += 1

            img = Image.open(myfile)
            img = img.resize((200, 200))
            if (np.asarray(img).shape == (200, 200)):
                continue
            elif (np.asarray(img).shape == (200, 200, 4)):
                continue
            
            
            training_data = np.append(training_data, np.array([np.asarray(img)]), axis=0)
training_data.shape
        

In [ ]:
training_data.shape
with zipfile.ZipFile(zip_path) as myzip:
    fileInitial = myzip.open(relevant_files[5274])
    img = Image.open(fileInitial)
    img = img.resize((200, 200))
img

In [ ]:
len(relevant_files)

In [ ]:
import pickle
filename = '../input/training-data/training_data.pickle'

    
training_data = None
with open(filename, 'rb') as infile:
    training_data = pickle.load(infile)

print(training_data.shape)

filename


In [ ]:
outname = '../input/notebook333c9b5d70/training_data_out'
a = None
import pickle
with open(outname, 'rb') as inputfile:
    a = pickle.load(inputfile)

a[0]


In [ ]:
import tensorflow as tf



import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display



BUFFER_SIZE = training_data.shape[0]
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(training_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
BUFFER_SIZE

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(50*50*256, use_bias=False, input_shape=(300,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((50,50, 256)))
    assert model.output_shape == (None, 50, 50, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 50, 50, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 100, 100, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    assert model.output_shape == (None, 200, 200, 3)

    return model
generator = make_generator_model()
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[200, 200, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(300, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model
discriminator = make_discriminator_model()


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

EPOCHS = 50
noise_dim = 300
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()


In [ ]:
train(train_dataset, EPOCHS)